## NHD Network Analysis Demo
___TL;DR___: *We are trying to parallelize hydraulic calculations for dynamic subsets of the U.S. river and stream network*<br><br>
The following was developed as part of the process of preparing a method for forecasting flows on the US network of rivers and streams as represented in the National Hydrography Dataset (NHD). The NHD is a continuously evolving characterization of a fractal system so we felt that we needed to plan to have some flexibility. We hope to identify the complexity inherent in the network at different levels of resolution and we hope to be able to do so dynamically. The goal is also to be able to manage the complexity calculation for arbitrary collections of headwater points, such as might be obtained from a list of named streams or during a major flood event in a particular region.<br>
As a point of terminology, we use the word 'routing' as shorthand to refer to the computation of the translation of a particular flow condition, high or low, to downstream (or in some cases upstream) areas of influence.
The network complexity is related to the potential for parallelization of a serial analysis of the network. We have identified three levels of parallelization that may be implemented: 
1. System-level parallelization of independent networks -- the routing computations for the Mississippi River have little (nothing, except conceptual similarity and a shared existence on earth) to do with the computations for the Columbia river for any practical level of analysis. The system of networks across the US is what we are considering in general.
1. Network-level parallelization of interconnected reaches -- There is a need to consider the computations for adjacent branches within a network of con-flowing streams, but with proper ordering, some of the computations may be considered in parallel. For example, the Illinois River headwaters and the Mississippi River headwaters are related within their broader Mississippi network, but the routing calculations for those headwaters are pratically agnostic to one another.
1. Reach-level parallelization of the specific routing computation -- the numerical work of routing water downstream is a matrix computation and consists of exploring solutions to differential equations, all of which may potentially be examined in parallel, under the proper conditions and with suitable assumptions.<br>


### Import the git repo including test data
This git repo is a fork/branch of the national water model public repository hosted by NCAR. The NCAR repo is the basis for the WRF-Hydro model that is presently the modeling engine of the [US National Water Model.](https://water.noaa.gov/about/nwm)<br>

The network analysis code assumes that the downstream neighbor is identified in the table for each stream segment as is the case for the test datasets. 

We recognize that others have done similar work and may possibly have done it better. We are working on being more able to nimbly respond to suggestions and opportunities for improvement. Please let us know if you see something we could do better or of course feel free to fork and improve what you see.

In [1]:
import sys

try:
    import google.colab

    ENV_IS_CL = True
    subprocess.run(
        [
            "git",
            "clone",
            "--single-branch",
            "--branch",
            "master",
            "https://github.com/NOAA-OWP/t-route.git",
        ]
    )
    sys.path.append("/content/src/python_framework_v01")
    subprocess.run(["pip", "install", "geopandas"])
    subprocess.run(["pip", "install", "netcdf4"])
    # default recursion limit (~1000) is slightly too small for the deepest branches of the network
    sys.setrecursionlimit(6000)
    # TODO: convert recursive functions to stack-based functions
except:
    ENV_IS_CL = False
    sys.path.append(r"../src/python_framework_v01")


### Some general functions and a test case
The next blocks use functions for several modules (`nhd_network_utilities` and `recursive_print`) from the git repository, to create the `connnections` objects which characterize the network to be analyzed.<br><br>
The `test_rows` object simulates a river network dataset such as we recieve from the National Hydrography Dataset. Each data row has a node ID, a 'to' node ID, and some other relevant data. For this test dataset, the second data column is a dummy length (and the last column could be some other value, but we haven't tried anything yet... stay tuned) and in our traversals, we can add up the lengths as a surrogate for more complex water routing functions we need to eventually manage.

In [2]:
import nhd_network_utilities_v01 as nnu
import recursive_print

# def main():
if 1 == 1:
    """##TEST"""
    print("")
    print("Executing Test")
    # Test data
    test_rows = [
        [50, 178, 51, 0],
        [51, 178, 50, 0],
        [60, 178, 61, 0],
        [61, 178, 62, 0],
        [62, 178, 60, 0],
        [70, 178, 71, 0],
        [71, 178, 72, 0],
        [72, 178, 73, 0],
        [73, 178, 70, 0],
        [80, 178, 81, 0],
        [81, 178, 82, 0],
        [82, 178, 83, 0],
        [83, 178, 84, 0],
        [84, 178, 80, 0],
        [0, 456, -999, 0],
        [1, 178, 4, 0],
        [2, 394, 0, 0],
        [3, 301, 2, 0],
        [4, 798, 0, 0],
        [5, 679, 4, 0],
        [6, 523, 0, 0],
        [7, 815, 2, 0],
        [8, 841, -999, 0],
        [9, 514, 8, 0],
        [10, 458, 9, 0],
        [11, 832, 10, 0],
        [12, 543, 11, 0],
        [13, 240, 12, 0],
        [14, 548, 13, 0],
        [15, 920, 14, 0],
        [16, 920, 15, 0],
        [17, 514, 16, 0],
        [18, 458, 17, 0],
        [19, 832, 18, 0],
        [20, 543, 19, 0],
        [21, 240, 16, 0],
        [22, 548, 21, 0],
        [23, 920, 22, 0],
        [24, 240, 23, 0],
        [25, 548, 12, 0],
        [26, 920, 25, 0],
        [27, 920, 26, 0],
        [28, 920, 27, 0],
    ]

    test_key_col = 0
    test_downstream_col = 2
    test_length_col = 1
    test_terminal_code = -999
    debuglevel = 0
    verbose = False

    (
        connections,
        all_keys,
        ref_keys,
        headwater_keys,
        terminal_keys,
        terminal_ref_keys,
        circular_keys,
        junction_keys,
        visited_keys,
        visited_terminal_keys,
        junction_count,
        confluence_segment_set,
        waterbody_dict,
        waterbody_segments,
        waterbody_outlet_set,
        waterbody_upstreams_set,
        waterbody_downstream_set,
    ) = nnu.build_connections_object(
        geo_file_rows=test_rows,
        key_col=test_key_col,
        mask_set={row[test_key_col] for row in test_rows},
        downstream_col=test_downstream_col,
        length_col=test_length_col,
        terminal_code=test_terminal_code,
        verbose=verbose,
        debuglevel=debuglevel,
    )
    # for index, value in enumerate(test_return_values):
    #     print(f"{index}. {str(value)[:100]} [{type(value).__name__}]")
    recursive_print.print_connections(
        headwater_keys=headwater_keys,
        down_connections=connections,
        up_connections=connections,
        terminal_code=test_terminal_code,
        terminal_keys=terminal_keys,
        terminal_ref_keys=terminal_ref_keys,
        debuglevel=debuglevel,
    )

    recursive_print.print_basic_network_info(
        connections=connections,
        headwater_keys=headwater_keys,
        junction_keys=junction_keys,
        terminal_keys=terminal_keys,
        terminal_code=test_terminal_code,
        verbose=verbose,
        debuglevel=debuglevel,
    )


# if __name__ == "__main__":
#     main()



Executing Test
########################
Downstream Connections
########################
1 with length 178
4 with length 798
0 with length 456
########################
3 with length 301
2 with length 394
0 with length 456
########################
5 with length 679
4 with length 798
0 with length 456
########################
6 with length 523
0 with length 456
########################
7 with length 815
2 with length 394
0 with length 456
########################
20 with length 543
19 with length 832
18 with length 458
17 with length 514
16 with length 920
15 with length 920
14 with length 548
13 with length 240
12 with length 543
11 with length 832
10 with length 458
9 with length 514
8 with length 841
########################
24 with length 240
23 with length 920
22 with length 548
21 with length 240
16 with length 920
15 with length 920
14 with length 548
13 with length 240
12 with length 543
11 with length 832
10 with length 458
9 with length 514
8 with length 841
###################

### Real Networks
(you can skip this cell to test the code on the simple case generated above...)

In [4]:
import os

if ENV_IS_CL:
    root = "/content/t-route-jsh/"
elif not ENV_IS_CL:
    root = os.path.dirname(os.path.abspath(""))
test_folder = os.path.join(root, r"test")
geo_input_folder = os.path.join(test_folder, r"input", r"geo")

supernetworks = {}
"""##NHD Subset (Brazos/Lower Colorado)"""
supernetworks.update({"Brazos_LowerColorado_ge5": {"data": None, "values": None}})
"""##NHD CONUS order 5 and greater"""
supernetworks.update({"CONUS_ge5": {"data": None, "values": None}})
"""These are large -- be careful"""
supernetworks.update({"CONUS_FULL_RES_v20": {"data": None, "values": None}})
supernetworks.update(
    {"CONUS_Named_Streams": {"data": None, "values": None}}
)  # create a subset of the full resolution by reading the GNIS field

debuglevel = -1
verbose = False

# The CONUS_ge5 and Brazos_LowerColorado_ge5 datasets are included
# in the github test folder and are extracts from the NHD version 1.2 datasets
# from https://www.nohrsc.noaa.gov/pub/staff/keicher/NWM_live/web/data_tools/
#
# The CONUS_FULL_RES file was generated from the RouteLink file in the parameter
# archive and converted to a compressed NetCDF via the following command:
# nccopy -d1 -s RouteLink_NWMv2.0_20190517_cheyenne_pull.nc RouteLink_NWMv2.0_20190517_cheyenne_pull.nc4s
# TODO: Explain CONUS_Named_Streams
# CONUS_Named_Streams was generated by intersecting the FULL_RES file ...
# of the data in the nohrsc-hosted archive but are too large to efficiently
# package inside of the repository.

for sn in supernetworks:
    print(sn)
    supernetworks[sn]["data"], supernetworks[sn]["values"] = nnu.set_networks(
        supernetwork=sn,
        geo_input_folder=geo_input_folder,
        verbose=verbose,
        debuglevel=debuglevel,
    )


Brazos_LowerColorado_ge5
reading -- dataset: /home/chp20/github/t-route/test/input/geo/Channels/NHD_BrazosLowerColorado_Channels.shp; layer: 0; driver: ESRI Shapefile
   OBJECTID_1  OBJECTID  featureID  linkDim     link  order_  Length       to  \
0        1499     25442    3764288   460086  3764288       6  2150.0  3764296   
1        1500     25443    3764296   460088  3764296       6  1277.0  3765756   
2        1501     25444    3766380   460092  3766380       6   431.0  3766382   
3        1502     25445    3765756   460090  3765756       6  1274.0  3766380   
4        1503     25447    3765796   460180  3765796       6   219.0  3765798   

     MusK  MusX   So     n  BtmWdth  ChSlp   Qi gages  NHDWaterbo      lat  \
0  3600.0   0.2  0.0  0.05  39.8032   0.05  0.0  None       -9999  29.0176   
1  3600.0   0.2  0.0  0.05  39.8543   0.05  0.0  None       -9999  29.0059   
2  3600.0   0.2  0.0  0.05  39.8653   0.05  0.0  None       -9999  28.9876   
3  3600.0   0.2  0.0  0.05  39.856

### With this, we can separate the different rivers in the network
Once a 'connection' object has been created with a representation of the river network, we can traverse that object and perform calculations -- in the example below, we parallelize the process of traversing the independent portions of the network and then serially compute the number of junctions. This corresponds to the "**System-level parallelization**" mentioned as _item 1_ above.
### NOW for the next step
We could compute total upstream length or (and this is the real goal) flow due to incoming lateral contributions from the land accumulated over the entire upstream network. That second calculation can also be parallelized but we have to figure out how to accomplish intelligently so that the collective calculation is network-aware. Such a parallelization would be the "**Network-level parallelization of interconnected reaches**" mentioned as _item 2_ above. The upstream length will depend on the number of upstream branches and their configuration, so there has to be some concept of stream order and topology built into the parallelization method.

In [7]:
# parallel compute
import time
import multiprocessing
from functools import partial

"""Test"""
# terminal_keys = test_return_values[4]
# circular_keys = test_return_values[6]
# terminal_keys_super = terminal_keys - circular_keys
# con = test_return_values[0]
# terminal_code = test_terminal_code
"""Streams of NHD order 5 or greater confluent to the Brazos and Lower Colorado"""
# terminal_keys = supernetworks['Brazos_LowerColorado_ge5']['values'][4]
# circular_keys = supernetworks['Brazos_LowerColorado_ge5']['values'][6]
# terminal_keys_super = terminal_keys - circular_keys
# con = supernetworks['Brazos_LowerColorado_ge5']['values'][0]
# terminal_code = supernetworks['Brazos_LowerColorado_ge5']['data']['terminal_code']
"""CONUS streams of NHD order 5 or greater"""
# terminal_keys = supernetworks['CONUS_ge5']['values'][4]
# circular_keys = supernetworks['CONUS_ge5']['values'][6]
# terminal_keys_super = terminal_keys - circular_keys
# con = supernetworks['CONUS_ge5']['values'][0]
# terminal_code = supernetworks['CONUS_ge5']['data']['terminal_code']
"""Full Res NHD"""
terminal_keys = supernetworks["CONUS_FULL_RES_v20"]["values"][4]
circular_keys = supernetworks["CONUS_FULL_RES_v20"]["values"][6]
terminal_keys_super = terminal_keys - circular_keys
con = supernetworks["CONUS_FULL_RES_v20"]["values"][0]
terminal_code = supernetworks["CONUS_FULL_RES_v20"]["data"]["terminal_code"]
"""Named Streams"""
# terminal_keys = supernetworks['CONUS_Named_Streams']['values'][4]
# circular_keys = supernetworks['CONUS_Named_Streams']['values'][6]
# terminal_keys_super = terminal_keys - circular_keys
# con = supernetworks['CONUS_Named_Streams']['values'][0]
# terminal_code = supernetworks['CONUS_Named_Streams']['data']['terminal_code']


def recursive_junction_read(
    keys, network, terminal_code=0, verbose=False, debuglevel=0
):
    global con
    for key in keys:
        ckey = key
        try:
            ukeys = con[key]["upstreams"]
            while not len(ukeys) >= 2 and not (ukeys == {terminal_code}):
                if debuglevel <= -3:
                    print(f"segs at ckey {ckey}: {network['segment_count']}")
                # the terminal code will indicate a headwater
                if debuglevel <= -4:
                    print(ukeys)
                (ckey,) = ukeys
                ukeys = con[ckey]["upstreams"]
            if ukeys == {terminal_code}:
                if debuglevel <= -3:
                    print(f"headwater found at {ckey}")
                network["segment_count"] += 1
                if debuglevel <= -3:
                    print(f"segs at ckey {ckey}: {network['segment_count']}")
            elif len(ukeys) >= 2:
                network["segment_count"] += 1
                if debuglevel <= -3:
                    print(f"junction found at {ckey} with upstreams {ukeys}")
                network["segment_count"] += 1
                if debuglevel <= -3:
                    print(f"segs at ckey {ckey}: {network['segment_count']}")
                network["junction_count"] += 1  # the Terminal Segment
                recursive_junction_read(
                    ukeys,
                    network,
                    terminal_code=terminal_code,
                    verbose=verbose,
                    debuglevel=debuglevel,
                )
                # print(ukeys)
                ukeys = con[ckey]["upstreams"]
                ckey = ukeys
        except:
            if debuglevel <= -2:
                print(f"There is a problem with connection: {key}: {con[key]}")


def network_trace(nid, terminal_code=terminal_code, verbose=False, debuglevel=0):

    network = {}
    global con
    us_length_total = 0

    if verbose:
        print(f"\ntraversing upstream on network {nid}:")
    # try:
    if 1 == 1:
        network.update({"junction_count": 0})
        network.update({"segment_count": 0})  # the Terminal Segment
        recursive_junction_read(
            [nid],
            network,
            verbose=verbose,
            terminal_code=terminal_code,
            debuglevel=debuglevel,
        )
        if verbose:
            print(f"junctions: {network['junction_count']}")
        if verbose:
            print(f"segments: {network['segment_count']}")
    # except Exception as exc:
    #     print(exc)
    # TODO: compute upstream length as a surrogate for the routing computation
    return {nid: network, "upstream_length": us_length_total}


TypeError: 'NoneType' object is not subscriptable

In [ ]:
###continuing from previous cell
networks = {
    terminal_key: {}
    for terminal_key in terminal_keys_super
    # # Toggle these comments to try removing/isolating some of the larger networks
    #                   if terminal_key in [
    #                   if terminal_key not in [
    #                           22811611 # Mississippi River, LA
    #                           , 23832907 # Columbia River, WA
    #                           , 626220 # Rio Grande, TX/MX
    #                           , 21412883 # Colorado River, AZ/MX
    #                           , 18524217 # Mobile River, AL
    #                           , 15183793 # Atchafalaya, LA
    #                   ]
}
debuglevel = -1
verbose = False

if verbose:
    print("verbose output")
if verbose:
    print(f"number of Independent Networks to be analyzed is {len(super_networks)}")
if verbose:
    print(f"Multi-processing will use {multiprocessing.cpu_count()} CPUs")
if verbose:
    print(f"debuglevel is {debuglevel}")

start_time = time.time()
results_serial = {}
for nid, network in networks.items():
    network.update(
        network_trace(
            nid, terminal_code=terminal_code, verbose=verbose, debuglevel=debuglevel
        )[nid]
    )
print("--- %s seconds: serial compute ---" % (time.time() - start_time))
if debuglevel <= -1:
    print(len(networks.items()))
if debuglevel <= -2:
    print(networks)

## Notice that I'm not timing the initialization in each case,
## which might be considered cheating a little bit.
## I timed it for the first case for reference.
nids = (nid for nid in networks)
start_time = time.time()
with multiprocessing.Pool() as pool:
    print(
        "--- %s seconds: parallel overhead to load multiprocessing.Pool() ---"
        % (time.time() - start_time)
    )
    start_time = time.time()
    results = pool.map(network_trace, nids)
    print(
        "--- %s seconds: parallel compute using default terminal_code ---"
        % (time.time() - start_time)
    )
if debuglevel <= -1:
    print(len(results))
if debuglevel <= -2:
    print(results)

nids = (nid for nid in networks)
snt = partial(network_trace, terminal_code=terminal_code)
with multiprocessing.Pool() as pool:
    start_time = time.time()
    results = pool.map(snt, nids)
    print(
        "--- %s seconds: parallel compute using partial function ---"
        % (time.time() - start_time)
    )
if debuglevel <= -1:
    print(len(results))
if debuglevel <= -2:
    print(results)

nidsWtc = ([nid, terminal_code] for nid in networks)
with multiprocessing.Pool() as pool:
    start_time = time.time()
    results = pool.starmap(network_trace, nidsWtc)
    print(
        "--- %s seconds: parallel compute with list of lists and starmap ---"
        % (time.time() - start_time)
    )
if debuglevel <= -1:
    print(len(results))
if debuglevel <= -2:
    print(results)


### Colab output

For the 14351 independent networks (and 2.7M segments) of the NWM Full Resolution dataset on a Google colaboratory VM, the algorithm steps through the segments of the different independent networks in order in about 4 seconds, regardless of the parallelization method. On a modest workstation at NWC with 12 cores, the compute time was approximately 2 seconds for a serial compute and 1.1 seconds for each of the parallel methods. Notably, because we are only breaking apart the independent networks, the maximum parallel speedup is limited by the size of the largest network -- the Mississippi River. By executing with the terminal node for the Mississippi River removed from the evaluation set, the serial calculation takes only 1.4 seconds and the parallel executions drop to approximately 0.4 seconds.<br>
```
--- 4.638037443161011 seconds: serial compute ---  
14351  
--- 0.0757453441619873 seconds: parallel overhead to load multiprocessing.Pool() ---  
--- 4.344968557357788 seconds: parallel compute using default terminal_code ---  
14351  
--- 4.323424577713013 seconds: parallel compute using partial function ---  
14351  
--- 4.167566537857056 seconds: parallel compute with list of lists and starmap ---  
14351  
```  